In [ ]:
import pennylane as qml
from qoop.core import ansatz, state
import pennylane.numpy as nps
import qiskit
import numpy as np
from scipy.stats import unitary_group
from qiskit.circuit.library import UnitaryGate
num_qubits = 3
dev = qml.device("default.qubit")

qc = qiskit.QuantumCircuit(num_qubits)

u = UnitaryGate(unitary_group.rvs(2**num_qubits))

qc.append(u, range(num_qubits))
qcx = qiskit.transpile(qc, basis_gates=['h','s','cx','u','rx','ry','rz'], optimization_level=3)

@qml.qnode(dev, diff_method="parameter-shift")
def circuit2(thetas):
    for i in range(0, len(thetas), 3):
        qml.RX(thetas[i], wires = 0)
        qml.RY(thetas[i+1], wires = 1)
        qml.RZ(thetas[i+2], wires = 2)
        qml.CNOT(wires = [0, 1])
        qml.CNOT(wires = [1, 2])
    qml.from_qiskit(qcx)
    return qml.expval(qml.PauliZ(0))
    

def cost(thetas):
    return 1 - circuit2(thetas)

thetas = nps.random.uniform(0, 2*nps.pi, 18, requires_grad=True)
steps = 10
costs = []
opt = qml.AdamOptimizer(stepsize = 0.1)
for n in range(steps):
    thetas, prev_cost = opt.step_and_cost(cost, thetas)
    if prev_cost < 10^(-6):
        print("Achieved error threshold at step", n)
        break
    costs.append(prev_cost)

0: ─╭●──────────────────╭RY(2.53)──RZ(3.57)──RX(1.27)──RZ(0.38)─╭●──────────────────╭RY(6.23)
1: ─╰RY(1.93)─╭●────────│──────────RZ(0.84)──RX(1.00)──RZ(6.13)─╰RY(1.72)─╭●────────│────────
2: ───────────╰RY(2.36)─╰●─────────RZ(1.44)──RX(3.79)──RZ(1.02)───────────╰RY(5.29)─╰●───────

───RZ(1.57)──RX(4.77)──RZ(3.48)─┤  
───RZ(1.38)──RX(1.73)──RZ(4.02)─┤  
───RZ(4.97)──RX(1.18)──RZ(1.29)─┤  
